In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

In [ ]:
diabetes = pd.read_csv('data/diabetes.csv')

In [ ]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = (diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min())

In [ ]:
drop_indices = np.random.choice(diabetes.index, 368, replace=False)
diabetes = diabetes.drop(drop_indices)

In [ ]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome',axis=1), diabetes['Outcome'], test_size=0.2)

In [ ]:
X_train[X_train == 0] = 0.000001
X_test[X_test == 0] = 0.000001

In [ ]:
X_train = X_train.values.tolist()
X_test = X_test.values.tolist()
y_train = y_train.values.tolist()
y_test = y_test.values.tolist()

In [ ]:
y_train = [int(x) for x in y_train]
y_test =  [int(x) for x in y_test]

In [ ]:
"""
with open('results/version4/train_data.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version4/test_data.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)
"""

In [2]:
with open('results/version4/train_data.json') as json_file:
    X_train = json.load(json_file)[0]

with open('results/version4/train_data.json') as json_file:
    y_train = json.load(json_file)[1]
    
with open('results/version4/test_data.json') as json_file:
    X_test = json.load(json_file)[0]

with open('results/version4/test_data.json') as json_file:
    y_test = json.load(json_file)[1]

## Test

In [ ]:
weights = quantumNeuronFIT(X_train, 
                           y_train,
                           init_weight = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
                           threshold=0.7,
                           lrParameter=0.1, 
                           n_epochs=20, 
                           trainingBias=False,
                           epoch_results=True,
                           phaseEstrategyOperator='original',
                           trainingApproaches={'hsgs','phase-encoding'})

In [ ]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.7,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=10,
                                   bias=False, 
                                   phaseEstrategyOperator='original',
                                   testingApproaches={'hsgs', 'phase-encoding'})

In [ ]:
print('phase avg_error', results[0])
print('phase weights', results[2])

print('\HSGS avg_error', results[3])
print('HSGS weights', results[5])

#print('phase neuron_outputs: ', results[1])
#print('HSGS neuron_outputs: ', results[4])
#print('targets: ', results[6])
#print('examples: ', results[7])

## Search

In [20]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [21]:
# search space

initial_weights = [
 [0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.50]
]

threshold_space = [0.5, 0.7, 0.9]
lr_space = [0.01, 0.1]
bias_space = [False, True]
strategy_space = ['angleradius', 'original', 'angle', 'radius']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=w,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=20, 
                                               trainingBias=bias,
                                               epoch_results=False,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs','phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=5,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'hsgs', 'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)

                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[3])
                    output['neuron_outputs'].append(results[4])
                    output['trained_weights'].append(results[5])
                    output['initial_weights'].append(w)
                    print('\n')


lrParameter:  0.01
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  angleradius

best error phase-encoding training:  118
best error HSGS training:  196
AVG TEST ERROR HSGS    0.675
AVG TEST ERROR PHASE   0.3625


lrParameter:  0.01
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  angleradius

best error phase-encoding training:  123
best error HSGS training:  196
AVG TEST ERROR HSGS    0.675
AVG TEST ERROR PHASE   0.325


lrParameter:  0.1
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  angleradius

best error phase-encoding training:  120
best error HSGS training:  196
AVG TEST ERROR HSGS    0.675
AVG TEST ERROR PHASE   0.325


lrParameter:  0.1
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  angleradius

best error phase-encoding training:  121
best error HSGS training:  196
AVG TEST ERROR HSGS    0.675
AVG TEST ERROR PHASE   0.325


lrParameter:  0.01
threshold:  0.7
trainingBias:  False
phaseEstrategyOperator:  angleradius

best 

In [25]:
experiment_data = pd.DataFrame(output)
experiment_data

,model,phase_strategy,bias,threshold,lr,avg_error,trained_weights,initial_weights,neuron_outputs
0,phase-encoding,angleradius,False,0.5,0.01,0.3625,"[-18.801269358517906, -14.832449508096847, -12...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,HSGS,angleradius,False,0.5,0.01,0.6750,"[8.217653858823546, 22.312060701507576, 22.560...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,phase-encoding,angleradius,True,0.5,0.01,0.3250,"[-19.423637883000463, -15.30776488013578, -13....","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,HSGS,angleradius,True,0.5,0.01,0.6750,"[8.217653858823546, 22.312060701507576, 22.560...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,phase-encoding,angleradius,False,0.5,0.10,0.3250,"[-192.99272713274064, -152.14516972230265, -13...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,HSGS,angleradius,False,0.5,0.10,0.6750,"[77.67653858823769, 218.6206070150749, 221.106...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,phase-encoding,angleradius,True,0.5,0.10,0.3250,"[-191.2592013372959, -151.15121934916914, -132...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,HSGS,angleradius,True,0.5,0.10,0.6750,"[77.67653858823769, 218.6206070150749, 221.106...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,phase-encoding,angleradius,False,0.7,0.01,0.3250,"[-19.43558858947985, -15.341089324211916, -13....","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,HSGS,angleradius,False,0.7,0.01,0.6750,"[8.217653858823546, 22.312060701507576, 22.560...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [26]:
experiment_data.to_csv('results/results_experiment_diabetes_angleradius2.csv')

In [38]:
def float_to_binary(x, m, n):
    """Convert the float value `x` to a binary string of length `m + n`
    where the first `m` binary digits are the integer part and the last
    'n' binary digits are the fractional part of `x`.
    """
    x_scaled = round(x * 2 ** n)
    return '{:0{}b}'.format(x_scaled, m + n)

print(float_to_binary(0.7, 0, 4))

1011
